Moodle XML Question Generator, (C) 2019-2020 Kosaku Nagasaka (Kobe University)

# how to use moodle_xqg for questions with graphs

An example of question generator with graphics by moodle_xqg (Moodle XML Question Generator for Python).

## loading the package

**moodle_xqg.core** is required mandatory. **moodle_xqgb.qbank.common** is not required but recommended since this includes some useful functions (e.g. deleting duplicate answers).

In [1]:
import moodle_xqg.core as mxqg
import moodle_xqg.qbank.common as mxqg_common

Mostly the following modules are necessary. For example, random integer generations and manipulating mathematical expressions are very common tasks.

In [2]:
import random
import sympy

## example of question generator with graphs

Your question generator must be defined as a child class of **Question**, and the class implements the following 5 methods: **question_generate**, **correct_answers_generate**, **incorrect_answers_generate**, **question_text** and **answer_text**.

Your question generator class has to generate the identifier of each quiz and store it in the variable **Quiz.quiz_identifier**. The package distinguish quizzes by this value. Therefore, the package generates only one quiz if you forget to implement this part.

The graph of function in the SVG format is generated by the **sympy_expr_to_svg** method that is defined in the class **moodle_xqg.qbank.common**.

In [3]:
class linear_polynomial_graph(mxqg.Question):
    def __init__(self, lower=-10, upper=10):
        # range of integers to be generated
        self.lower = lower
        self.upper = upper
        # variable and function
        self.x = sympy.symbols('x')
        self.correct_f = 0
    def question_generate(self, _quiz_number=0):
        quiz = mxqg.Quiz(name='graph of linear function', quiz_number=_quiz_number, lang='en')
        # generates a quiz data (exclude 0 since SymPy can not convert 0 to LaTeX)
        quiz.data = [random.choice(list(set(range(self.lower, self.upper + 1))-{0})) for i in range(2)]
        quiz.quiz_identifier = hash('{}*self.x+{}'.format(quiz.data[0], quiz.data[1]))
        # generates the correct choice
        self.correct_f = quiz.data[0]*self.x+quiz.data[1]
        ans = { 'fraction': 100, 'data': mxqg_common.sympy_expr_to_svg(self.correct_f) }
        quiz.answers.append(ans)
        return quiz        
    def correct_answers_generate(self, quiz, size=1):
        # nothing to do since we already have the correct answer generated
        pass
    def incorrect_answers_generate(self, quiz, size=4):
        answers = []
        ans = { 'fraction': 0 }
        # wrong sign of slope 
        f = -quiz.data[0]*self.x+quiz.data[1]
        if f != self.correct_f:
            ans['data'] = mxqg_common.sympy_expr_to_svg(f)
            ans['feedback'] = 'The sign of the slope is not correct.'
            answers.append(dict(ans))
       # wrong sign of y-intercept
        f = quiz.data[0]*self.x-quiz.data[1]
        if f != self.correct_f:
            ans['data'] = mxqg_common.sympy_expr_to_svg(f)
            ans['feedback'] = 'The y-intercept is the constant term.'
            answers.append(dict(ans))
        # wrong signs of slope and y-intercept
        f = -quiz.data[0]*self.x-quiz.data[1]
        if f != self.correct_f:
            ans['data'] = mxqg_common.sympy_expr_to_svg(f)
            ans['feedback'] = 'Check the definitions of slope and y-intercept.'
            answers.append(dict(ans))
        # wrong y-intercept
        f = quiz.data[0]*(self.x + quiz.data[1] - 
                          sympy.Rational(quiz.data[1],quiz.data[0]))+quiz.data[1]
        if f != self.correct_f:
            ans['data'] = mxqg_common.sympy_expr_to_svg(f)
            ans['feedback'] = 'The y-intercept is the constant term.'
            answers.append(dict(ans))
        if len(answers) >= size:
            return random.sample(answers,k=size)
        return answers    
    def question_text(self, quiz):
        return 'Choose the graph of \( y=f(x) \) where ' + \
                mxqg_common.sympy_expr_to_text(quiz.data[0]*self.x+quiz.data[1], prefix='f(x)=') + '.'
    def answer_text(self, ans):
        return ans['data']

To generate actual quizzes, the **generate** method is used. However, before calling the method, we have to generate an instance of the question generator. The number of quizzes to be generated is specified by **size=100** for *generate* (in the default, 10 quizzes will be generated).

In [4]:
lpg = linear_polynomial_graph()
lpg_quizzes = mxqg.generate(lpg, category='graph example')

The result of question generation is an instance of the **Quizzes** class. We can use the methods of the class as follows.

In [5]:
lpg_quizzes.preview(size=2)

Finally, the **save** method writes the quizzes in the specified file in the Moodle XML format that can be imported to Moodle.

In [6]:
lpg_quizzes.save('linear_polynomial_graph_in_english.xml')

Moreover, if you do not need the quiz id in the question text, specify **show_quiz_number=False** as an additional option to *save*.